# Imports & Settings


In [2]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from deep_translator import GoogleTranslator
import re 
from math import isnan

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [3]:
# dtype={'type': str} prevents being confused with data type for large data sets
train = pd.read_csv('data/train.csv', index_col='id', dtype={'type': str})
test = pd.read_csv('data/test.csv', index_col='id', dtype={'type': str})
train_translated = pd.read_csv('data/train_translated.csv', dtype={'type': str})
test_translated = pd.read_csv('data/test_translated.csv', index_col='id', dtype={'type': str})
combined_data = pd.read_csv('data/combined_data.csv', index_col='id', dtype={'type': str})
combined_data_translated = pd.read_csv('data/combined_data_translated.csv', index_col='id', dtype={'type': str})
combined_data_fully_translated = pd.read_csv('data/combined_data_fully_translated.csv', index_col='id', dtype={'type': str})
prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})

/tmp/ipykernel_6336/1699356385.py:9: DtypeWarning: Columns (13,266) have mixed types. Specify dtype option on import or set low_memory=False.
  prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})


# Parameter & Unit & Value

# Finish unit tranlation/ unification &  values to float

In [184]:
data = prep.copy()

In [194]:
data['value'] = data['value'].apply(lambda x: float(x.replace(',', '.')) if type(x) == str else x)

In [196]:
data.to_csv('data/prep.csv')

In [209]:
data = prep.copy()

In [210]:
# e.g wrong translation
data['unit'] = data['unit'].replace('e.g','mm')
data['unit'] = data['unit'].replace('pc','pieces')
data['unit'] = data['unit'].replace('page','pages')
# unify units
data['unit'] = data['unit'].replace('10 x 15 cm','100 x 150 mm')

In [211]:
data.to_csv('data/prep.csv')

# mm to cm

In [296]:
data = prep.copy()

In [297]:
data['value'] = data.apply(lambda item: item['value'] / 10 if item['unit'] == 'mm' else item['value'], axis=1)

In [298]:
data['unit'] = data['unit'].replace('mm','cm')

In [299]:
data['value'] = pd.to_numeric(data['value'])

In [301]:
data.to_csv('data/prep.csv')

# Combine parameter, unit & w/h values to value

In [303]:
data = prep.copy()

In [304]:
data['unit'] = data['unit'].replace(np.nan,'*')
data['parameter'] = data['parameter'].replace(np.nan,'*')

In [305]:
def get_squared(item):
    if ' x ' in item:
        return item + '²'
    else:
        return item

In [306]:
data['unit'] = data['unit'].apply(lambda x: get_squared(x))

In [307]:
def extract_width_height_from_unit_to_value(item):
    unit = item[0]
    value = item[1]
    if ' x ' in unit:
        split = unit.split(' ')
        x = split[0]
        y = split[2]
        real_unit = split[3]
        real_value = [x,y]        
        return [real_unit, real_value]

    else:
        return [unit, value]

In [308]:
# execution order is important
data['value'] = data.apply(lambda item: extract_width_height_from_unit_to_value(item[['unit','value']])[1], axis=1)
data['unit'] = data.apply(lambda item: extract_width_height_from_unit_to_value(item[['unit','value']])[0], axis=1)


In [309]:
data['parameter_and_unit'] = data['parameter'] + ' IN ' + data['unit']

In [313]:
data.to_csv('data/prep.csv')

# parameter_and_units as single features with respective values

In [315]:
data = prep.copy()

In [316]:
# parameter_and_unit turned into one hot encoded features
data= pd.get_dummies(data, columns=['parameter_and_unit'], prefix='', prefix_sep='')

In [317]:
data.to_csv('data/prep.csv')

In [ ]:
# now put values into one hot encoded features

In [319]:
data = prep.copy()

In [320]:
# for all new "parameter with unit" columns put the value in the column where a 1 is - others are 0 and remain 0
for column in data.columns:
    if ' IN ' in column:
        data[column] = data.apply(lambda item: item['value'] * item[column], axis=1)


In [322]:
data.to_csv('data/prep.csv')

# make all size features numeric

In [4]:
data = prep.copy()

In [18]:
def extract_height_width(item):    
    if type(item) == str:
        # Converting string to list
        res = item.strip('][').split(', ')
        h = float(res[0].strip('\''))
        w = float(res[1].strip('\''))
        return [h,w]

    else:
        return [float('nan'), float('nan')]

In [19]:
for column in data.columns:
    # all the parameter with unit columns that contain arrays that are represeted as strings
    if (' IN ' in column) and (data[column].dtype == object):
        data[column + '_height'] = data.apply(lambda item: extract_height_width(item[column])[0], axis=1)
        data[column + '_width'] = data.apply(lambda item: extract_height_width(item[column])[1], axis=1)

In [26]:
for column in data.columns:
    if (' IN ' in column):
        data[column] = data[column].replace(np.nan,0)

In [28]:
data.to_csv('data/prep.csv')

# some more